In [4]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import *
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import sklearn.ensemble
from sklearn.utils import parallel_backend
from joblibspark import register_spark
from sklearn.metrics import mean_absolute_error

In [5]:
spark = SparkSession.builder.appName("Vu dep trai").getOrCreate()
# conf = pyspark.SparkConf().setMaster("spark://node-master:7077")\
#         .setAppName("Vu dep trai")\
#         .set("spark.executor.memory","15g")
# # sc = SparkContext.getOrCreate(conf=conf)
# # spark.stop()
# sc = SparkContext(conf = conf)
# spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/01 11:12:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/01 11:12:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
df_stores_raw = spark.read.csv("data/ba-walmart/stores.csv", header=True, inferSchema=True)
df_feature_raw = spark.read.csv("data/ba-walmart/features.csv", header=True, inferSchema=True)
df_train_raw = spark.read.csv("data/ba-walmart/train.csv", header=True, inferSchema=True)
df_test_raw = spark.read.csv("data/ba-walmart/test.csv", header=True, inferSchema=True)

In [22]:
df_feature = df_feature_raw.drop("MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5")
df = df_train_raw.join(df_feature, how="left", on=["Store", "Date", "IsHoliday"], ).join(df_stores_raw, how="left", on=["Store"])
df_test = df_test_raw.join(df_feature, how="left", on=["Store", "Date", "IsHoliday"]).join(df_stores_raw, how="left", on=["Store"])
df = df.withColumn("CPI", df["CPI"].cast(FloatType())).withColumn("Unemployment", df["Unemployment"].cast(FloatType()))
df_test = df_test.withColumn("CPI", df_test["CPI"].cast(FloatType())).withColumn("Unemployment", df_test["Unemployment"].cast(FloatType()))


In [23]:
df = df.withColumn("Year", year("Date")).withColumn("Month", month("Date")).withColumn("Week", weekofyear("Date"))
df_test = df_test.withColumn("Year", year("Date")).withColumn("Month", month("Date")).withColumn("Week", weekofyear("Date"))
df = df.withColumn("IsHoliday", df["IsHoliday"].cast(IntegerType()))
df_test = df_test.withColumn("IsHoliday", df_test["IsHoliday"].cast(IntegerType()))

In [24]:
df_clean = df.filter(df["Weekly_Sales"] > 0)
df_clean = df_clean.filter(df_clean["Weekly_Sales"] < 450000)

In [25]:
types = df_clean.select("Type").distinct().collect()
types.sort()
mapping = {t.Type: str(i) for i, t in enumerate(types)}
df_clean = df_clean.replace(mapping, subset=["Type"])
df_test = df_test.replace(mapping, subset=["Type"])
df_train = df_clean.withColumn("Type", df_clean["Type"].cast(IntegerType()))
df_test = df_test.withColumn("Type", df_test["Type"].cast(IntegerType()))

In [32]:
## From EDA select important columns
input_col = ['Store', 'IsHoliday', 'Type', 'Size', 'Week','Dept','Year']
target = ['Weekly_Sales']

In [33]:
# all date in train set
total_date = df_train_clean.select("Date").collect()
total_date.sort()
total_date = [d.Date for d in total_date]
break_date = total_date[int(len(total_date)*0.8)]
break_date

datetime.datetime(2012, 4, 13, 0, 0)

In [34]:
# split train and validation set
df_train = df_clean.filter(df_clean["Date"] < break_date)
df_valid = df_clean.filter(df_clean["Date"] >= break_date)

In [35]:
df_train_pd = df_train.toPandas()
df_valid_pd = df_valid.toPandas()

/opt/bitnami/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/opt/bitnami/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [36]:
register_spark()

rnd = sklearn.ensemble.RandomForestRegressor()

# grid_param = {
#     'max_depth': [25, 30, 35],
#     'max_features': [1.0, 0.9, 0.8],
#     'max_samples': [0.5, 0.75, 1.0]
# }

# simulate grid search
grid_param = {
    'max_depth': [25],
    'max_features': [0.8],
    'max_samples': [0.5, 0.75]
}


with parallel_backend('spark'):
    grid_search = GridSearchCV(estimator=rnd,
                               param_grid=grid_param,
                               cv=3,
                               n_jobs=-1,
                               verbose=2)
    grid_search.fit(df_train_pd[input_col], df_train_pd[target])


Fitting 3 folds for each of 2 candidates, totalling 6 fits


/opt/bitnami/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/bitnami/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/bitnami/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/bitnami/python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: 

23/02/01 11:32:54 ERROR Executor: Exception in task 0.0 in stage 81.0 (TID 134)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3745)
	at java.base/java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:120)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:95)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:156)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1849)
	at java.base/java.io.ObjectOutputStream.write(ObjectOutputStream.java:708)
	at org.apache.spark.util.Utils$.writeByteBuffer(Utils.scala:244)
	at org.apache.spark.scheduler.DirectTaskResult.$anonfun$writeExternal$1(TaskResult.scala:53)
	at org.apache.spark.scheduler.DirectTaskResult$$Lambda$2668/0x0000000841157840.apply$mcV$sp(Unknown Source)
	at scala.runtime.java8.JFunction0$

Exception in thread "task-result-getter-3" java.lang.InterruptedException
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1040)
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1345)
	at scala.concurrent.impl.Promise$DefaultPromise.tryAwait(Promise.scala:242)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:258)
	at scala.concurrent.impl.Promise$DefaultPromise.result(Promise.scala:263)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:293)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:103)
	at org.apache.spark.storage.BlockManager.fetchRemoteManagedBuffer(BlockManager.scala:1154)
	at org.apache.spark.storage.BlockManager.$anonfun$getRemoteBlock$8(BlockManager.scala:1098)
	at scala.Option.orElse(Option.scala:447)
	at org.apache.spark.storage.Block

23/02/01 11:32:54 ERROR ChunkFetchRequestHandler: Error sending result ChunkFetchSuccess[streamChunkId=StreamChunkId[streamId=59744389015,chunkIndex=0],buffer=org.apache.spark.storage.BlockManagerManagedBuffer@60bae10f] to /172.17.0.2:38168; closing connection
io.netty.channel.StacklessClosedChannelException
	at io.netty.channel.AbstractChannel.close(ChannelPromise)(Unknown Source)
23/02/01 11:32:54 ERROR ChunkFetchRequestHandler: Error sending result ChunkFetchSuccess[streamChunkId=StreamChunkId[streamId=59744389016,chunkIndex=0],buffer=org.apache.spark.storage.BlockManagerManagedBuffer@5f86e2d7] to /172.17.0.2:38168; closing connection
io.netty.channel.StacklessClosedChannelException
	at io.netty.channel.AbstractChannel.close(ChannelPromise)(Unknown Source)
23/02/01 11:32:54 ERROR TransportRequestHandler: Error sending result RpcResponse[requestId=5906802833204407975,body=NioManagedBuffer[buf=java.nio.HeapByteBuffer[pos=0 lim=13 cap=13]]] to /172.17.0.2:38168; closing connection
io.n

...

ConnectionRefusedError: [Errno 111] Connection refused

Error in callback <function SparkDistributedBackend.start_call.<locals>.on_post_run_cell at 0x7f5dc0e8c280> (for post_run_cell):


ConnectionRefusedError: [Errno 111] Connection refused

Error in callback <function SparkDistributedBackend.start_call.<locals>.on_post_run_cell at 0x7f5dc2164940> (for post_run_cell):


ConnectionRefusedError: [Errno 111] Connection refused